In [134]:
import requests
import sys, os
from os import path
sys.path.append('/Users/admin/Desktop/个人项目/anjuke_spider/')
import json
import time
from lxml import etree
import pandas as pd

## 访问小区页

https://shanghai.anjuke.com/community/

In [56]:
url = 'https://shanghai.anjuke.com/community/'
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'cookie':'sessid=E5EC0E4B-4C48-5B5C-EA77-0440C24A6AA9; aQQ_ajkguid=CCE441E6-9D82-1788-8FBA-0CC0FF482D49; ctid=11; twe=2; wmda_uuid=d639d05f241b74c8beb2a345090a7fd0; wmda_visited_projects=%3B6289197098934; id58=CrIHp2KUeFJ3BwnFBDASAg==; _ga=GA1.2.195237907.1653897298; _gid=GA1.2.1335426594.1653897298; 58tj_uuid=eaa6df61-18e8-4312-aa47-cf0957ff79eb; als=0; isp=true; fzq_h=a8eafab09d9ac1214696944d51bfec63_1653897320816_e289a17d57a74bba842cb3323c952e81_1709567291; seo_source_type=0; ajk-appVersion=; fzq_js_anjuke_ershoufang_pc=d238625fb6060c04469e1290eecaf2fc_1653901810905_23; wmda_session_id_6289197098934=1653902105642-7a60deb2-a3d4-3a6c; init_refer=; new_uv=2; new_session=0; obtain_by=2; ajk_member_verify=hAjDLO%2BYv6cH48rAMRWcG5I9XasyuqFQJw2W5QBmVbQ%3D; ajk_member_verify2=MjQ5NDcyMzQ3fHM2bXhLcnB8MQ%3D%3D; fzq_js_anjuke_xiaoqu_pc=44116584dff4e49396a76958d4b4a52a_1653902184987_25; ajkAuthTicket=TT=829e0f92da645d9b3485dcff94af291f&TS=1653902185446&PBODY=bBYOIqzmDkmfSozXHfcqw52GMfhLdSaaaHf6GSYWhWbUEHmPXKnES1fphveH8j0E2pWLYSTckhR5NgSPUGLTSFtMO3ZsvP_APhExnjS8gWtC2G1eEMyXwPPHFJ9y9mN7IUQkclJkBxsSh1QnawYbMAOsmGAEWK2Si0ABL4TtJb4&VER=2&CUID=P0ki0_oMTt8NAdOd0tDoYwj5svAO2gEh; xxzl_cid=7e414a50acbe4a57b67cf17d00b35ce1; xzuid=3f01ada0-a8d8-48ee-a0fc-fbc3c396b549'}

res = requests.get(url=url,headers=headers)
tree = etree.HTML(res.text)

## 获取城区

In [71]:
region_list = []
for li in tree.xpath('/html/body/div/div/div/section/section[2]/div[1]/section/div/ul/li')[1:]:
    temp={}
    temp['region_url'] = li.xpath('./a/@href')[0]
    temp['region_name'] = li.xpath('./a/text()')[0]
    region_list.append(temp)

In [72]:
# with open('/Users/admin/Desktop/个人项目/anjuke_spider/cache/region_list.json','w') as fp:
#     json.dump(region_list,fp)

## 获取街道

In [86]:
neighbor_list = []
for i in region_list: #跳过第一个
    res = requests.get(url=i['region_url'],headers=headers)
    time.sleep(1)
    tree = etree.HTML(res.text)
    for li in tree.xpath('/html/body/div/div/div/section/section[2]/div[1]/section/div/div[2]/li')[1:]:
        temp={}
        temp['neighbor_url'] = li.xpath('./a/@href')[0]
        temp['neighbor_name'] = li.xpath('./a/text()')[0]
        neighbor_list.append(temp)

In [88]:
# with open('/Users/admin/Desktop/个人项目/anjuke_spider/cache/neighbor_list.json','w') as fp:
#     json.dump(neighbor_list,fp)

In [242]:
with open('/Users/admin/Desktop/个人项目/anjuke_spider/cache/neighbor_list.json','r') as fp:
    a = json.load(fp)
print(len(a))

312


## 解析搜索页函数

In [156]:
def parse_search_page(tree):
    page_res = []
    for a in tree.xpath('/html/body/div/div/div/section/section[3]/section/div[2]/a'):
        temp = {}
        temp['url']= a.xpath('./@href')[0]
        temp['title']=a.xpath('.//div[@class="li-title"]/div/text()')[0]
        temp['year_address']=';'.join(a.xpath('.//div[@class="props nowrap"]//text()'))
        temp['tags'] = ';'.join(a.xpath('.//div[@class="prop-tags"]/span/text()'))
        try:
            temp['price'] = a.xpath('.//div[@class="community-price"]/strong/text()')[0]
        except Exception as e:
            temp['price'] = ''
        page_res.append(temp)
    return page_res

In [117]:
search_page_res = []

In [120]:
for i in neighbor_list[1:]:
    res = requests.get(url=i['neighbor_url'],headers=headers)
    tree = etree.HTML(res.text)
    page_res = parse_search_page(res)
    search_page_res += page_res
    break

## 测试爬取

In [177]:
search_page_res = []
for single_neighbor in neighbor_list[:1]:
    res = requests.get(url= single_neighbor['neighbor_url'], headers=headers)
    tree = etree.HTML(res.text)
    page_res = []
    for a in tree.xpath('/html/body/div/div/div/section/section[3]/section/div[2]/a'):
        temp = {}
        temp['url']= a.xpath('./@href')[0]
        temp['title']=a.xpath('.//div[@class="li-title"]/div/text()')[0]
        temp['year_address']=';'.join(a.xpath('.//div[@class="props nowrap"]//text()'))
        temp['tags'] = ';'.join(a.xpath('.//div[@class="prop-tags"]/span/text()'))
        try:
            temp['price'] = a.xpath('.//div[@class="community-price"]/strong/text()')[0]
        except Exception as e:
            temp['price'] = ''
        page_res.append(temp)
    search_page_res += page_res        

In [178]:
len(page_res)

17

## 解析详情页

In [181]:
for i in search_page_res:
    res = requests.get(url=i['url'],headers=headers)
    tree = etree.HTML(res.text)
    break

In [240]:
detail_info = {}

for div in tree.xpath('//div[@class="info"]/div[@class="info-list"]/div'):
    temp = {}
    label = div.xpath('./div[@class="label"]/text()')[0]
    try:
        value = div.xpath('./div[@class="hover"]/div[1]/text()')[0].replace('\n','').replace(' ','')
    except Exception as e:
        value = ''.join(div.xpath('./div[2]//text()')).replace('\n','').replace(' ','')
    
    temp[label]= value

    detail_info.update(**temp) 

In [241]:
detail_info

{'物业类型': '公寓住宅',
 '权属类别': '商品房住宅',
 '竣工时间': '2004年',
 '产权年限': '70年',
 '总户数': '421户',
 '总建面积': '暂无',
 '容积率': '2.88',
 '绿化率': '43.0%',
 '建筑类型': '高层',
 '所属商圈': '花木',
 '统一供暖': '否',
 '供水供电': '民用',
 '停车位': '287(1:0.7)',
 '物业费': '3.50元/平米/月',
 '停车费': '地面150.00元/月',
 '车位管理费': '暂无',
 '物业公司': '业主自管',
 '小区地址': '东绣路66弄',
 '开发商': '单位自建'}

## 获取失败的url